In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
import json
from privacypacking.utils.utils import load_logs
import pandas as pd
from experiments.ray.analysis import load_tasks, load_ray_experiment, load_latest_ray_experiment, load_latest_scheduling_results, load_latest_scheduling_results, load_latest_ray_experiment, load_scheduling_queue
import plotly.express as px
from privacypacking.budget.curves import  LaplaceCurve, GaussianCurve, SubsampledGaussianCurve, SyntheticPolynomialCurve
from privacypacking.budget import Budget, Task, Block
from privacypacking.schedulers.metrics import OverflowRelevance, FlatRelevance
from privacypacking.budget.block_selection import RandomBlocks
from privacypacking.utils.plot import plot_budgets
import yaml
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

from omegaconf import OmegaConf
from pathlib import Path
from collections import defaultdict

In [ ]:
from privacypacking.budget import ALPHAS
from privacypacking.utils.utils import add_workload_args_to_results

Let's see what curves are allocated by different schedulers

In [ ]:
df = load_latest_ray_experiment()
# df = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2022-04-07_09-58-57"))
# df = load_latest_ray_experiment("/home/pierre/privacypacking/logs/ray/run_and_report_2022-04-07_10-07-26")



In [ ]:
df.columns

In [ ]:
df = add_workload_args_to_results(df)

In [ ]:
df.describe()

df.describe

In [ ]:
fig = px.line(
    # rdf.query("data_lifetime == 50 and max_blocks == 400").sort_values("T"),
    df.query("epsilon_min_std == 0 and range_std == 0 and range_min == 0.005").sort_values(["total_tasks", "alpha_std"]),
    x="alpha_std",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    markers=True,
    facet_col="epsilon_min_avg",
    facet_row="total_tasks",
)
fig.update_yaxes(rangemode="tozero")
fig

In [ ]:
fig = px.line(
    # rdf.query("data_lifetime == 50 and max_blocks == 400").sort_values("T"),
    df.sort_values(["total_tasks", "alpha_std"]),
    x="alpha_std",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    markers=True,
    # facet_col="data_lifetime",
    facet_row="total_tasks",
)
fig.update_yaxes(rangemode="tozero")
fig

In [ ]:
# df = load_latest_scheduling_results(expname="ray/run_and_report_2022-04-05_22-11-57")
# TODO: custom load from Zoo